In [70]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import json_normalize

In [71]:
#Kolonner i datasettet som er av typen Json, og må flates ut.
json_columns = [
    'device', 'geoNetwork', 'totals', 'trafficSource'
]
train_path ='../oblig_4/data/train.csv'

In [72]:
load_train = pd.read_csv(train_path, converters={column: json.loads for column in json_columns},
                    dtype={'fullVisitorId':'str'})

In [73]:
# Kilde: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.json_normalize.html?highlight=json_normalize
# Kilde: https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields
# Kilde: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html?highlight=merge#pandas.DataFrame.merge
def flatten_json(df, json_columns):
    
    for column in json_columns:
        # Normaliserer json-data til en flat tabell.
        column_as_ch = json_normalize(df[column])
        
        # Navngir de nye subkolonnene som kommer av normaliseringen.
        column_as_ch.columns = [f"{column}_{subcolumn}" for subcolumn in column_as_ch.columns]
        
        # Dropper den gamle kolonnen, fletter de nye kolonnene inn i chunken.
        df = df.drop(column, axis=1).merge(column_as_ch, right_index=True, left_index=True)
    return df

In [74]:
train = flatten_json(load_train, json_columns) #Henter datasett og transformerer json-kolonner.

In [75]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 903653 entries, 0 to 903652
Data columns (total 55 columns):
 #   Column                                             Non-Null Count   Dtype 
---  ------                                             --------------   ----- 
 0   channelGrouping                                    903653 non-null  object
 1   date                                               903653 non-null  int64 
 2   fullVisitorId                                      903653 non-null  object
 3   sessionId                                          903653 non-null  object
 4   socialEngagementType                               903653 non-null  object
 5   visitId                                            903653 non-null  int64 
 6   visitNumber                                        903653 non-null  int64 
 7   visitStartTime                                     903653 non-null  int64 
 8   device_browser                                     903653 non-null  object
 9   devi

In [76]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device_browser,device_browserVersion,...,trafficSource_adwordsClickInfo.criteriaParameters,trafficSource_isTrueDirect,trafficSource_referralPath,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adContent,trafficSource_campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,not available in demo dataset,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
train.isnull().sum()

channelGrouping                                           0
date                                                      0
fullVisitorId                                             0
sessionId                                                 0
socialEngagementType                                      0
visitId                                                   0
visitNumber                                               0
visitStartTime                                            0
device_browser                                            0
device_browserVersion                                     0
device_browserSize                                        0
device_operatingSystem                                    0
device_operatingSystemVersion                             0
device_isMobile                                           0
device_mobileDeviceBranding                               0
device_mobileDeviceModel                                  0
device_mobileInputSelector              

In [78]:
train['trafficSource_source'].unique() #Eksempel på irrelevant data.

array(['google', 'baidu', 'googleweblight.com', 'sites.google.com',
       'bing', 'siliconvalley.about.com', 'analytics.google.com',
       'Partners', '(direct)', 'mall.googleplex.com', 'google.com',
       'google.co.in', 'qiita.com', 'google.pl', 'mail.google.com',
       'hangouts.google.com', 'dealspotr.com', 'optimize.google.com',
       'arstechnica.com', 'dfa', 'docs.google.com', 'sashihara.jp',
       'google.gatewaycdi.com', 'phandroid.com', 'images.google',
       'youtube.com', 'l.facebook.com', 'facebook.com', 'quora.com',
       'm.facebook.com', 'reddit.com', 'google.co.jp', 'aol',
       'moma.corp.google.com', 'blog.golang.org', 'yahoo', 't.co',
       'nordic.businessinsider.com', 'duckduckgo.com', 'm.baidu.com',
       'tpc.googlesyndication.com', 'googleads.g.doubleclick.net',
       '0.shared.bow.cat2.ads-bow.vu.borg.google.com:9856',
       'online-metrics.com', 'cv.il3.ub.edu', 'gophergala.com',
       'plus.google.com', 's0.2mdn.net', 'keep.google.com',
       

In [79]:
train['totals_transactionRevenue'].unique()

array([nan, '37860000', '306670000', ..., '335260000', '30750000',
       '69390000'], dtype=object)

In [80]:
from pycaret.regression import *

In [81]:
#Initialiser pycaret.
#exp_reg = setup(train, target='totals_transactionRevenue')